In [1]:
import numpy as np
import tensorflow as tf

# Create dataset from numpy
np_data=np.random.sample((100,2))
numpy_dataset=tf.data.Dataset.from_tensor_slices(np_data)


# Create dataset from tensor
tensor_data=tf.random_uniform([100,2])
tensor_dataset=tf.data.Dataset.from_tensor_slices(tensor_data)

# Print the first element of the dataset using Iterator
def printDataset(dataset):
  iterator=dataset.make_initializable_iterator()
  element=iterator.get_next()

  with tf.Session() as sess:
    sess.run(iterator.initializer)
    print(sess.run(element))

print("\nThe first element of the numpy_dataset:")
printDataset(numpy_dataset)
print("\nThe first element of the tensor_dataset:")
printDataset(tensor_dataset)


The first element of the numpy_dataset:
Instructions for updating:
Colocations handled automatically by placer.
[0.02523586 0.04324205]

The first element of the tensor_dataset:
[0.99107265 0.05335832]


In [2]:
x_data=np.arange(0,10)
y_data=np.arange(50,60)

# Create dataset from numpy
x_dataset=tf.data.Dataset.from_tensor_slices(x_data)
y_dataset=tf.data.Dataset.from_tensor_slices(y_data)

# Zip two dataset
train_dataset=tf.data.Dataset.zip((x_dataset,y_dataset))

# Create batches with size=3
train_dataset=train_dataset.batch(batch_size=3)

print("\nThe first run:")
printDataset(train_dataset)
print("\nThe second run:")
printDataset(train_dataset)

# Shuffle the dataset
train_dataset=train_dataset.shuffle(buffer_size=100)
print("\nAfter shuffling data:")
print("The first run:")
printDataset(train_dataset)
print("\nThe second run:")
printDataset(train_dataset)


The first run:
(array([0, 1, 2]), array([50, 51, 52]))

The second run:
(array([0, 1, 2]), array([50, 51, 52]))

After shuffling data:
The first run:
(array([0, 1, 2]), array([50, 51, 52]))

The second run:
(array([3, 4, 5]), array([53, 54, 55]))


In [27]:
# Example of using One-shot Iterator
one_shot_dataset=tf.data.Dataset.range(10)

# Create an one_shot iterator
iterator=one_shot_dataset.make_one_shot_iterator()
next_element=iterator.get_next()

sess=tf.Session()
for i in range(10):
  print(sess.run(next_element),end=' ')

# OutOfRangeError occurs as Iterator already reaches the end of the dataset
print(sess.run(next_element))

0 1 2 3 4 5 6 7 8 9 

OutOfRangeError: ignored

In [19]:
# Example of using Initializable Iterator
max_value=tf.placeholder(tf.int64)
init_dataset=tf.data.Dataset.range(max_value)

# Create an initializable iterator
iterator=init_dataset.make_initializable_iterator()
next_element=iterator.get_next()

# Create a tensorflow session
sess=tf.Session()

# Create a dataset with 5 data
sess.run(iterator.initializer, feed_dict={max_value:5})
print('Dataset with 5 data: ')
for i in range(5):
  print(sess.run(next_element),end=' ')

# Create a dataset with 8 data
sess.run(iterator.initializer, feed_dict={max_value:8})
print('\n\nDataset with 8 data: ')
for i in range(8):
  print(sess.run(next_element),end=' ')

Dataset with 5 data: 
0 1 2 3 4 

Dataset with 8 data: 
0 1 2 3 4 5 6 7 

In [23]:
# Example of using Reinitializable Iterator

# Create two datasets with the same structure 
train_dataset=tf.data.Dataset.range(10)
test_dataset=tf.data.Dataset.range(5)

# Create a reinitializable iterator defined by its structure
iterator=tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
next_element=iterator.get_next()

# Create initialization operation for each dataset 
train_init_op=iterator.make_initializer(train_dataset)
test_init_op=iterator.make_initializer(test_dataset)

# Create a tensorflow session
sess=tf.Session()

# Get the train data
sess.run(train_init_op)
print('Train dataset: ')
for i in range(10):
  print(sess.run(next_element),end=' ')

# Switch from train_dataset to test_dataset
sess.run(test_init_op)
print('\n\nTest dataset: ')
for i in range(5):
  print(sess.run(next_element),end=' ')

Train dataset: 
0 1 2 3 4 5 6 7 8 9 

Test dataset: 
0 1 2 3 4 

In [30]:
# Example of using Feedable Iterator

# Create two datasets with the same structure 
train_dataset=tf.data.Dataset.range(10)
test_dataset=tf.data.Dataset.range(5)

# Create a feedable iterator defined by a placeholder and its structure
handle=tf.placeholder(tf.string)
iterator=tf.data.Iterator.from_string_handle(handle,train_dataset.output_types,train_dataset.output_shapes)
next_element=iterator.get_next()

# Create one-shot iterator for train_dataset 
train_iterator=train_dataset.make_one_shot_iterator()

# Create initializable iterator for test_dataset 
test_iterator=test_dataset.make_initializable_iterator()

# Create a tensorflow session
sess=tf.Session()

# Create handle for train_iterator and test_iterator
train_handle=sess.run(train_iterator.string_handle())
test_handle=sess.run(test_iterator.string_handle())


# Get the train data using train_iterator
print('Train dataset(using one-shot iterator): ')
for i in range(10):
  print(sess.run(next_element, feed_dict={handle:train_handle}),end=' ')

# Switch from one-shot iterator to initializable iterator
sess.run(test_iterator.initializer)
print('\n\nTest dataset(using initializable iterator): ')
for i in range(5):
  print(sess.run(next_element, feed_dict={handle:test_handle}),end=' ')

Train dataset(using one-shot iterator): 
0 1 2 3 4 5 6 7 8 9 

Test dataset(using initializable iterator): 
0 1 2 3 4 

In [58]:
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load and processing the data
wine_data = datasets.load_wine()
data=wine_data['data']
min_max_scaler = preprocessing.MinMaxScaler()
data_scaled = min_max_scaler.fit_transform(data)

#Split into train and test set
train_data, test_data, train_label, test_label = train_test_split(data_scaled,wine_data['target'],test_size=0.2)

# Create a dataset from train_data,train_label
tf.reset_default_graph() 
epoches=50
train_dataset=tf.data.Dataset.from_tensor_slices((train_data,train_label)).repeat(epoches).batch(len(train_data))

# Create an one-shot iterator
iterator = train_dataset.make_one_shot_iterator()
inputData, actual_labels = iterator.get_next()

# Create a neural network
hidden1=tf.layers.dense(inputData,128,activation=tf.nn.relu,name="layer1")
hidden2=tf.layers.dense(hidden1,64,activation=tf.nn.relu,name="layer2")
logit=tf.layers.dense(hidden2,3,name="logit")

# Define the loss function and Optimization algorithm
cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=train_label,logits=logit)
loss=tf.reduce_mean(cross_entropy,name="loss")
training_op=tf.train.AdamOptimizer(0.01).minimize(loss)    

# Calculate the accuracy of the model
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logit,1),actual_labels),tf.float64))

# Create a session and initialse all variables                       
sess=tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(epoches):
  # Train the model and calculate the accuracy of the current model
  _,train_accuracy=sess.run([training_op,accuracy])
  
  # Display the accuracy result
  print(i+1,'. Train accuracy: ',train_accuracy)

1 . Train accuracy:  0.23943661971830985
2 . Train accuracy:  0.6338028169014085
3 . Train accuracy:  0.704225352112676
4 . Train accuracy:  0.823943661971831
5 . Train accuracy:  0.9507042253521126
6 . Train accuracy:  0.9436619718309859
7 . Train accuracy:  0.9577464788732394
8 . Train accuracy:  0.971830985915493
9 . Train accuracy:  0.971830985915493
10 . Train accuracy:  0.971830985915493
11 . Train accuracy:  0.9788732394366197
12 . Train accuracy:  0.9788732394366197
13 . Train accuracy:  0.9788732394366197
14 . Train accuracy:  0.9859154929577465
15 . Train accuracy:  0.9859154929577465
16 . Train accuracy:  0.9929577464788732
17 . Train accuracy:  0.9929577464788732
18 . Train accuracy:  0.9929577464788732
19 . Train accuracy:  0.9929577464788732
20 . Train accuracy:  0.9929577464788732
21 . Train accuracy:  0.9929577464788732
22 . Train accuracy:  0.9929577464788732
23 . Train accuracy:  1.0
24 . Train accuracy:  1.0
25 . Train accuracy:  1.0
26 . Train accuracy:  1.0
27 . Tr